<a href="https://colab.research.google.com/github/yuktachikate/Financial-Dispute-AI-Assistant/blob/main/Financial_Dispute_AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio


In [5]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import random

# Automatically install missing dependencies
try:
    import gradio as gr
except ModuleNotFoundError:
    os.system("pip install gradio")
    import gradio as gr


In [6]:
# Step 1: Load Sample Financial Transaction Dispute Data
data = {
    "Customer_ID": [201, 202, 203, 204],
    "Transaction_Issue": [
        "Unauthorized transaction on credit card",
        "Incorrect interest charged on loan",
        "Double withdrawal detected",
        "Unrecognized charge from a vendor"
    ],
    "Previous_Resolution": [
        "Refund initiated",
        "Interest adjusted",
        "Duplicate charge reversed",
        "Charge disputed and refunded"
    ]
}
df = pd.DataFrame(data)
print("Sample Financial Transaction Dispute Data:")
display(df)


Sample Financial Transaction Dispute Data:


,Customer_ID,Transaction_Issue,Previous_Resolution
0,201,Unauthorized transaction on credit card,Refund initiated
1,202,Incorrect interest charged on loan,Interest adjusted
2,203,Double withdrawal detected,Duplicate charge reversed
3,204,Unrecognized charge from a vendor,Charge disputed and refunded


from matplotlib import pyplot as plt
df['Customer_ID'].plot(kind='hist', bins=20, title='Customer_ID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Transaction_Issue').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Previous_Resolution').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Transaction_Issue'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Customer_ID', y='Transaction_Issue', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['Previous_Resolution'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Customer_ID', y='Previous_Resolution', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(df['AI_Resolution'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(df, x='Customer_ID', y='AI_Resolution', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [7]:
# Step 2: Load Pre-trained LLM for Financial Dispute Resolution
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [8]:
# Step 3: Generate AI-Powered Dispute Resolutions
def generate_financial_response(issue):
    input_text = f"Customer issue: {issue}. Provide a professional resolution response complying with financial regulations."
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

In [9]:
# Step 4: Generate AI Responses for the Dataset
df["AI_Resolution"] = df["Transaction_Issue"].apply(generate_financial_response)

# Display Results
print("Generated AI Responses for Financial Transaction Issues:")
display(df)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generated AI Responses for Financial Transaction Issues:


,Customer_ID,Transaction_Issue,Previous_Resolution,AI_Resolution
0,201,Unauthorized transaction on credit card,Refund initiated,Customer issue: Unauthorized transaction on cr...
1,202,Incorrect interest charged on loan,Interest adjusted,Customer issue: Incorrect interest charged on ...
2,203,Double withdrawal detected,Duplicate charge reversed,Customer issue: Double withdrawal detected. Pr...
3,204,Unrecognized charge from a vendor,Charge disputed and refunded,Customer issue: Unrecognized charge from a ven...


In [10]:
# Step 5: AI Chatbot for Financial Dispute Resolution
def chatbot_response(user_query):
    response = generate_financial_response(user_query)
    return response

interface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(placeholder="Enter your financial dispute issue..."),
    outputs=gr.Textbox(),
    title="Financial Dispute AI Assistant",
    description="This AI Assistant provides automated resolutions for customer financial transaction disputes."
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0f08de66d1a7c2ce52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
# Step 6: Fraud Detection Using AI (Simulated)
def detect_fraud(transaction_amount, location, time_of_day):
    risk_score = random.randint(1, 100)
    if risk_score > 75:
        return f"Transaction flagged as HIGH RISK. Investigate immediately. Risk Score: {risk_score}"
    elif risk_score > 50:
        return f"Transaction flagged as MEDIUM RISK. Manual review recommended. Risk Score: {risk_score}"
    else:
        return f"Transaction appears safe. Risk Score: {risk_score}"

# Example Fraud Detection Test Cases
print(detect_fraud(5000, "New York", "Midnight"))
print(detect_fraud(100, "Los Angeles", "Afternoon"))


Transaction flagged as HIGH RISK. Investigate immediately. Risk Score: 88
Transaction flagged as HIGH RISK. Investigate immediately. Risk Score: 78
